In [69]:
import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os.path
import os

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [70]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65331%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=OkE85YlbAcdcll8ZmHXh4CpbEezPvX&access_type=offline
Descargando industrial8.csv: 100%
Descargando industrial9.csv: 100%
Descargando industrial10.csv: 100%
Descargando industrial7.csv: 100%
Descargando industrial6.csv: 100%
Descargando industrial4.csv: 100%
Descargando industrial3.csv: 100%
Descargando industrial2.csv: 100%
Descargando industrial5.csv: 100%
Descargando industrial1.csv: 100%
Descarga completa
Archivos descargados:
industrial1.csv
industrial10.csv
industrial2.csv
industrial3.csv
industrial4.csv
industrial5.csv
industrial6.csv
industrial7.csv
industrial8.csv
industrial9.csv


In [71]:
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['variable4_sistemas', 'variable5_sistemas', 'variable6_sistemas']
    },
    'sistemas': {
        '1': ['variable1_industrial', 'variable2_industrial', 'variable3_industrial'],
        '2': ['variable4_industrial', 'variable5_industrial', 'variable6_industrial']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}

In [72]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="1"

In [73]:
datos = cargar_datos(carrera, semestre)

columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD,RENDIMIENTO_UNO
0,474,65,73,75,79,20,2
1,523,99,69,78,60,1,2
2,483,63,76,82,77,20,1
3,528,88,69,78,75,19,1
4,478,78,69,66,69,8,2
...,...,...,...,...,...,...,...
2003,342,68,0,0,58,11,2
2004,347,69,0,0,71,19,2
2005,348,67,0,0,68,11,1
2006,340,70,0,0,63,11,2


In [74]:
X=df.loc[:, ~df.columns.str.contains('RENDIMIENTO')]
Y = df.loc[:, df.columns.str.contains('RENDIMIENTO')]                                                      
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2008, 6) (2008, 1)


In [75]:
X = X.astype('float32')                         
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2008, 6) (2008,)


In [76]:
from collections import Counter
frecuencias = Counter(Y)

# Imprimir las frecuencias
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 1 aparece 1249 veces
El valor 0 aparece 675 veces
El valor 2 aparece 83 veces
El valor 3 aparece 1 veces


### TRANSFORMACIÓN DE DATOS

In [77]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.598 -0.897  1.005  1.046  1.268  1.353]
 [ 2.502  2.983  0.97   1.072 -0.304 -2.363]
 [ 1.767 -1.135  1.031  1.104  1.099  1.353]
 [ 2.592  1.759  0.97   1.072  0.931  1.202]
 [ 1.673  0.621  0.97   0.965  0.432 -0.682]]


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD
0,1.597922,-0.896989,1.005440,1.046458,1.268133,1.353436
1,2.501767,2.983213,0.970075,1.071632,-0.304398,-2.362881


In [78]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
Xpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y))


Distribución de clases después del oversampling: Counter({1: 1249, 0: 1249, 2: 1249, 3: 1249})


In [79]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 3497 datos para entrenamiento y 1499 datos para prueba


In [80]:
from collections import Counter
frecuencias = Counter(Y_trn)

# Imprimir las frecuencias
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 0 aparece 861 veces
El valor 1 aparece 878 veces
El valor 3 aparece 862 veces
El valor 2 aparece 896 veces


### KNEIGHBORSCLASSIFIER

In [81]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    # Definir parámetros posibles del algoritmo
    parameters = {
        'n_neighbors': [i for i in range(14, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto'],
        'p': [i for i in range(1, 6)],
        'weights': ['uniform']
    }
    modelo = KNeighborsClassifier()

    # Validación cruzada
    semilla = 5
    num_folds = 10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    # Optimizar parámetro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)

    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)

    # Entrenar el modelo con los mejores hiperparámetros
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)

    return mejor_modelo
# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 67.8284076954564
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}


In [82]:


# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_knn.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.676907199934602
Exhaustividad (Recall):  0.6877918612408273
Puntuación F1:  0.6638923984240015
Exactitud:  0.6877918612408273

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.57      0.52      0.54       388
           1       0.53      0.28      0.37       371
           2       0.61      0.96      0.74       353
           3       0.99      1.00      1.00       387

    accuracy                           0.69      1499
   macro avg       0.67      0.69      0.66      1499
weighted avg       0.68      0.69      0.66      1499



### SVC

In [83]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 'kernel':  ['rbf'], 
            'C': [i/10000 for i in range(8,12,1)],
            'max_iter':[i for i in range(1,3,1)],
            'gamma' : [i/100 for i in range(90,110,5)],
            'random_state':[i for i in range(1,5,1)]}
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)



Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 49.729267294310276
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 1.05, 'kernel': 'rbf', 'max_iter': 2, 'random_state': 1}


In [84]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_svc.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.45564624316737384
Exhaustividad (Recall):  0.5210140093395597
Puntuación F1:  0.43950678419503864
Exactitud:  0.5210140093395597

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.34      0.88      0.49       388
           1       0.44      0.14      0.22       371
           2       0.00      0.00      0.00       353
           3       1.00      1.00      1.00       387

    accuracy                           0.52      1499
   macro avg       0.45      0.51      0.43      1499
weighted avg       0.46      0.52      0.44      1499



### DECISION TREE

In [85]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {          
              'max_depth':[i for i in range(3,6,1)],
              'min_samples_split' :  [i for i in range(1,3,1)],  
              'min_samples_leaf' : [i for i in range(1,3,1)], 
              'max_features' : [i for i in range(5,7,1)], 
              'splitter': ["best", "random"],
              'random_state': [i for i in range(1,4,1)],
              'criterion': ['entropy']}
    modelo = DecisionTreeClassifier()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 59.59533360622184
Mejor valor PARAMETRO usando k-fold: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'random_state': 1, 'splitter': 'best'}


In [86]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_tree.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.6029539378181916
Exhaustividad (Recall):  0.5843895930620414
Puntuación F1:  0.5517984311744568
Exactitud:  0.5843895930620414

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.55      0.16      0.25       388
           1       0.40      0.35      0.37       371
           2       0.44      0.84      0.58       353
           3       1.00      1.00      1.00       387

    accuracy                           0.58      1499
   macro avg       0.60      0.59      0.55      1499
weighted avg       0.60      0.58      0.55      1499



### NAIVE BAYES

In [87]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 54.50331559557922
Mejor valor PARAMETRO usando k-fold: {}


In [88]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_gaussian.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.5375310079011705
Exhaustividad (Recall):  0.5383589059372915
Puntuación F1:  0.5370029315409418
Exactitud:  0.5383589059372915

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.41      0.39      0.40       388
           1       0.33      0.30      0.31       371
           2       0.38      0.45      0.41       353
           3       1.00      1.00      1.00       387

    accuracy                           0.54      1499
   macro avg       0.53      0.53      0.53      1499
weighted avg       0.54      0.54      0.54      1499



### LDA

In [89]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
              'n_components':[1,2,3,4,5,None],
              'shrinkage': ['auto',None, 0, 0.001, 0.01, 0.1, 0.5,1]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 53.188374948833406
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': 0.5, 'solver': 'lsqr'}


In [90]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_LDA.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.4783646225090162
Exhaustividad (Recall):  0.5036691127418279
Puntuación F1:  0.48645341742314113
Exactitud:  0.5036691127418279

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.39      0.34      0.37       388
           1       0.28      0.20      0.23       371
           2       0.36      0.46      0.40       353
           3       0.86      1.00      0.93       387

    accuracy                           0.50      1499
   macro avg       0.47      0.50      0.48      1499
weighted avg       0.48      0.50      0.49      1499



### BAGGINGCLASSIFIER

In [91]:
from sklearn.ensemble import BaggingClassifier
estimator = BaggingClassifier()
estimator.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [92]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
              'max_samples' : [i/100.0 for i in range(40,50,5)],
             'max_features': [i/100.0 for i in range(60,65,5)],
              'bootstrap': [True], 
              'bootstrap_features': [True]}
    
    base_estimator= DecisionTreeClassifier(criterion= 'gini', 
                                max_depth=5, 
                                max_features= 3,
                                min_samples_leaf= 4, 
                                min_samples_split = 8,
                                random_state= 10, 
                                splitter= 'random')

    modelo = BaggingClassifier(estimator=base_estimator, oob_score=True, random_state=1)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 63.91297584936554
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.6, 'max_samples': 0.4, 'n_estimators': 755}


In [93]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_BG.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.8336205625040424
Exhaustividad (Recall):  0.8372248165443629
Puntuación F1:  0.8348755768974344
Exactitud:  0.8372248165443629

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       388
           1       0.70      0.64      0.67       371
           2       0.94      1.00      0.97       353
           3       1.00      1.00      1.00       387

    accuracy                           0.84      1499
   macro avg       0.83      0.84      0.84      1499
weighted avg       0.83      0.84      0.83      1499



### EXTRA TREES CLASSIFIER

In [94]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 
                'min_samples_split' : [i for i in range(1,3,1)], 
                'min_samples_leaf' : [i for i in range(0,2,1)] 
                  
              }
    
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40,
                                max_features=1,
                                max_depth= 10,
                                min_samples_leaf=1,  
                                min_samples_split = 2,
                                bootstrap=True,
                                criterion='gini') 

    #modelo = BaggingClassifier(estimator=base_estimator, oob_score=True, random_state=1)
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 76.49578387228816
Mejor valor PARAMETRO usando k-fold: {'min_samples_leaf': 1, 'min_samples_split': 2}


In [95]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_extra.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.8814728920458235
Exhaustividad (Recall):  0.8812541694462975
Puntuación F1:  0.8809593694668211
Exactitud:  0.8812541694462975

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.80      0.74      0.77       388
           1       0.75      0.79      0.77       371
           2       0.99      1.00      0.99       353
           3       1.00      1.00      1.00       387

    accuracy                           0.88      1499
   macro avg       0.88      0.88      0.88      1499
weighted avg       0.88      0.88      0.88      1499



### ADABOOST

In [96]:
from sklearn.ensemble import AdaBoostClassifier            
estimator  = AdaBoostClassifier ()
estimator.get_params()

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [97]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 
                 'learning_rate' : [i/10000.0 for i in range(5,20,5)]
           #  'learning_rate' : [i/10000.0 for i in range(45,55,1)],
              }
    
    semilla=7            
    modelo = AdaBoostClassifier(
                 estimator = None,  
                 algorithm = 'SAMME.R', 
                 random_state= None,
                  n_estimators =50) 

    #modelo = BaggingClassifier(estimator=base_estimator, oob_score=True, random_state=1)
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_ADA = entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 46.898567335243555
Mejor valor PARAMETRO usando k-fold: {'learning_rate': 0.0005}


In [98]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_ADA.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.3139640213736319
Exhaustividad (Recall):  0.4876584389593062
Puntuación F1:  0.36121288975721194
Exactitud:  0.4876584389593062

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       388
           1       0.00      0.00      0.00       371
           2       0.32      0.97      0.48       353
           3       0.93      1.00      0.96       387

    accuracy                           0.49      1499
   macro avg       0.31      0.49      0.36      1499
weighted avg       0.31      0.49      0.36      1499



### GRADIENT BOOSTING MACHINE

In [99]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 
                'subsample' : [ 0.5,0.6,0.7 , 0.75 , 0.8 , 0.85 , 0.9 , 0.95 , 1 ]        
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,
                                    learning_rate= 0.1,
                                    max_depth= 2,
                                    min_samples_split= 2,
                                    min_samples_leaf= 3,                           
                                    max_features= 2)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 65.65616045845272
Mejor valor PARAMETRO usando k-fold: {'subsample': 0.7}


In [100]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_GD.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))


Precisión:  0.7352946757418451
Exhaustividad (Recall):  0.7444963308872582
Puntuación F1:  0.7348615826829304
Exactitud:  0.7444963308872582

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.62      0.55      0.58       388
           1       0.59      0.49      0.53       371
           2       0.73      0.95      0.83       353
           3       1.00      1.00      1.00       387

    accuracy                           0.74      1499
   macro avg       0.73      0.75      0.73      1499
weighted avg       0.74      0.74      0.73      1499



### XGBOOST

In [101]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {
            'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
            'reg_lambda':  [i/100.0 for i in range(130,150,5)],
                }

             # 'n_estimators': [i for i in range(500,1000,100)]}
             # 'min_split_loss' : [0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1],  
             # 'objective' : ('binary:logistic', 'Multi: softprob')}
    semilla=7


    modelo = XGBClassifier (random_state=semilla,                       
                         n_estimators=40,
                         colsample_bytree = 1, 
                          subsample =1,
                           max_depth =2,
                           min_child_weight =6,
                           gamma = 0.05,
                          learning_rate = 0.3
                           )
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 64.85616045845272
Mejor valor PARAMETRO usando k-fold: {'reg_alpha': 0, 'reg_lambda': 1.35}


In [102]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_XB.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.8329523447111445
Exhaustividad (Recall):  0.8365577051367579
Puntuación F1:  0.8342226918782596
Exactitud:  0.8365577051367579

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       388
           1       0.69      0.63      0.66       371
           2       0.94      1.00      0.97       353
           3       1.00      1.00      1.00       387

    accuracy                           0.84      1499
   macro avg       0.83      0.84      0.84      1499
weighted avg       0.83      0.84      0.83      1499



### CATBOOST

In [103]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {
              'border_count':[53],
               'l2_leaf_reg': [42]} 
              #'iterations' : [i for i in range(100,1000,100)], 
               #'learning_rate' : [0.001, 0.005,0.01,0.05, 0.1, 0.2, 0.3, 0.4,0.5,1]}
              #'depth' :[3,4,5,6,7,8,10,13,15,17, None]}        
            # 'leaf_estimation_method': [ 'Newton' ,  'Gradient' , 'Exact' ],
           # 'bootstrap_type' : [ 'Bayesian' ,  'Bernoulli' , 'MVS', None ],
            #  'loss_function' : [ 'RMSE' ,'MAE ' , 'Logloss', 'CrossEntropy', 'MultiClass', None ],
             # 'ctr_border_count':[5,10,20,50,100,200],}

             # 'n_estimators': [i for i in range(500,1000,100)]}
             # 'min_split_loss' : [0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1],  
             # 'objective' : ('binary:logistic', 'Multi: softprob')}
    semilla=7


    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 73.89259107654522
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'l2_leaf_reg': 42}
0:	learn: 1.3626469	total: 3.68ms	remaining: 3.67s
1:	learn: 1.3407146	total: 6.97ms	remaining: 3.48s
2:	learn: 1.3207645	total: 10.4ms	remaining: 3.45s
3:	learn: 1.3016048	total: 14.1ms	remaining: 3.5s
4:	learn: 1.2834608	total: 17.7ms	remaining: 3.52s
5:	learn: 1.2662616	total: 21ms	remaining: 3.48s
6:	learn: 1.2493513	total: 24ms	remaining: 3.4s
7:	learn: 1.2343051	total: 27.8ms	remaining: 3.45s
8:	learn: 1.2191719	total: 31.8ms	remaining: 3.5s
9:	learn: 1.2056132	total: 35.3ms	remaining: 3.5s
10:	learn: 1.1924244	total: 38.7ms	remaining: 3.48s
11:	learn: 1.1799842	total: 41.7ms	remaining: 3.43s
12:	learn: 1.1681294	total: 45.1ms	remaining: 3.43s
13:	learn: 1.1573514	total: 48.4ms	remaining: 3.41s
14:	learn: 1.1458809	total: 51.5ms	remaining: 3.38s
15:	learn: 1.1363479	total: 55.2ms	remaining: 3.4s
16:	learn: 1.

In [104]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_CB.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.7227170724727544
Exhaustividad (Recall):  0.7344896597731821
Puntuación F1:  0.723817301339158
Exactitud:  0.7344896597731821

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.59      0.48      0.53       388
           1       0.55      0.51      0.53       371
           2       0.75      0.97      0.85       353
           3       1.00      1.00      1.00       387

    accuracy                           0.73      1499
   macro avg       0.72      0.74      0.72      1499
weighted avg       0.72      0.73      0.72      1499



### LIGHT GBM

In [105]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = { 
                  'min_child_samples' : [i for i in range(200, 10000, 100)]}
    semilla=7


    modelo = LGBMClassifier (random_state=semilla,                           
                        num_leaves =  10 ,
                        max_depth = 1   ,
                         
                        n_estimators = 100,    
                        learning_rate = 0.1 ,
                        class_weight=  None  ,
                        subsample = 1 ,
                        colsample_bytree= 1 ,
                        reg_alpha=  0,
                        reg_lambda = 0,
                        min_split_gain = 0,
                        boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = KFold(n_splits=num_folds, random_state=semilla, shuffle=True)

    #Optimizar parametro mediante GridSearchCV
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000920 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 394
[LightGBM] [Info] Number of data points in the train set: 3497, number of used features: 6
[LightGBM] [Info] Start training from score -1.401566
[LightGBM] [Info] Start training from score -1.382014
[LightGBM] [Info] Start training from score -1.361720
[LightGBM] [Info] Start training from score -1.400405
Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 58.364060581252566
Mejor valor PARAMETRO usando k-fold: {'min_child_samples': 600}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000218 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 394
[LightGBM] [Info] Number of data points in the train set: 3497, number of used features: 6
[LightGBM] [Info] Start training from score -1.401

In [106]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_LIGHT.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.6267177467891922
Exhaustividad (Recall):  0.628418945963976
Puntuación F1:  0.6182202726597111
Exactitud:  0.628418945963976

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.55      0.40      0.46       388
           1       0.42      0.35      0.38       371
           2       0.52      0.76      0.62       353
           3       1.00      1.00      1.00       387

    accuracy                           0.63      1499
   macro avg       0.62      0.63      0.62      1499
weighted avg       0.63      0.63      0.62      1499



### VOTING - VOTACIÓN DURA

In [107]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, learning_rate = 0.01, max_depth = 2,
                                       max_features =2, min_samples_leaf = 9, min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                        max_depth=8, max_features=1, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_samples_leaf=1, min_samples_split=5,
                                        min_weight_fraction_leaf=0.0,
                                        random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, algorithm ='SAMME.R', learning_rate = 0.001, 
                                random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                bootstrap= True, bootstrap_features = False, max_features = 0.9,
                                max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')

    modelo7 = XGBClassifier(n_estimators= 40, 
                      random_state=semilla,
                     colsample_bytree = 1, 
                          subsample =1,
                           reg_alpha = 0.2, 
                            reg_lambda= 1.35,
                           learning_rate= 0.3,
                           max_depth =2,
                           min_child_weight =6,
                           gamma = 0.05)
    
    
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                ('Bagging',modelo5),('Decision tree',modelo6),('XGB',modelo7)],
    voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_hard = entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 70.00425706099058  % 


In [108]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_voting_hard.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.695432156535864
Exhaustividad (Recall):  0.6964643095396932
Puntuación F1:  0.6899159807755835
Exactitud:  0.6964643095396932

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.61      0.43      0.50       388
           1       0.50      0.54      0.52       371
           2       0.67      0.82      0.74       353
           3       1.00      1.00      1.00       387

    accuracy                           0.70      1499
   macro avg       0.69      0.70      0.69      1499
weighted avg       0.70      0.70      0.69      1499



### VOTING - VOTACIÓN SUAVE

In [109]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = KFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(random_state=semilla, n_estimators=800, learning_rate = 0.01, max_depth = 2,
                                    max_features =2, min_samples_leaf = 9, min_samples_split = 2, subsample = 1 )
    
    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                        max_depth=8, max_features=1, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_samples_leaf=1, min_samples_split=5,
                                        min_weight_fraction_leaf=0.0,
                                        random_state=5, splitter='random')

    modelo2 = AdaBoostClassifier(estimator=base_estimator,n_estimators=1200, algorithm ='SAMME.R', learning_rate = 0.001, 
                                random_state=semilla)

    modelo3 = ExtraTreesClassifier(n_estimators=300, max_features=None,bootstrap = False, max_depth = 11,min_samples_split = 4, 
                                min_samples_leaf = 1)

    modelo4 = RandomForestClassifier (n_estimators=1100, max_features=None,
                                    min_samples_split = 5, max_depth= 3, min_samples_leaf= 1)

    model = DecisionTreeClassifier(criterion= 'gini', 
                                    max_depth=6, 
                                    max_features= None, 
                                    min_samples_leaf= 9, 
                                    min_samples_split = 2, 
                                    random_state= 10, 
                                    splitter= 'random')

    modelo5 = BaggingClassifier(estimator=model, n_estimators=50, random_state=semilla,
                                bootstrap= True, bootstrap_features = False, max_features = 0.9,
                                max_samples= 0.95)

    modelo6 = DecisionTreeClassifier(criterion= 'gini', max_depth=6,max_features= None, min_samples_leaf= 9, 
                                    min_samples_split = 2, random_state= 10, splitter= 'random')
    

    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7, 0.9,0.9, 0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 78.72410970118707  % 


In [110]:

# Predecir las etiquetas para los datos de prueba
Y_pred = modelo_voting_soft.predict(X_tst)

# Calcular la precisión
precision = precision_score(Y_tst, Y_pred, average='weighted')

# Calcular la exhaustividad (recall)
recall = recall_score(Y_tst, Y_pred, average='weighted')

# Calcular la puntuación F1
f1 = f1_score(Y_tst, Y_pred, average='weighted')

# Calcular la exactitud
accuracy = accuracy_score(Y_tst, Y_pred)

# Imprimir las métricas
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)

# Informe de clasificación detallado
print("\nInforme de clasificación:")
print(classification_report(Y_tst, Y_pred))

Precisión:  0.7766719684037725
Exhaustividad (Recall):  0.7805203468979319
Puntuación F1:  0.7759904048730465
Exactitud:  0.7805203468979319

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.68      0.56      0.62       388
           1       0.61      0.63      0.62       371
           2       0.81      0.93      0.86       353
           3       1.00      1.00      1.00       387

    accuracy                           0.78      1499
   macro avg       0.78      0.78      0.78      1499
weighted avg       0.78      0.78      0.78      1499

